In [8]:
from langchain.memory import ConversationBufferWindowMemory

# 대화의 특정부분 즉 제일 최근 부분만 저장한다는 특성을 가지고있다. 저장범위는 직접 설정 할 수 있다. 범위를 벗어난 부분은 제거된다.
# 장점: 모든 대화 내용을 저장하지 않아도 된다. 메모리를 특정 크기로 유지할 수 있다는게 큰 장점이다.
# 단점: 챗봇이 전체 대화가 아닌 최근 대화에만 집중

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [15]:
add_message(10, 1)

In [16]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='6'),
  AIMessage(content='1'),
  HumanMessage(content='7'),
  AIMessage(content='1'),
  HumanMessage(content='8'),
  AIMessage(content='1'),
  HumanMessage(content='9'),
  AIMessage(content='1'),
  HumanMessage(content='10'),
  AIMessage(content='1')]}